In [ ]:
# !pip install kaggle

# !kaggle --version

# !kaggle --help

# !kaggle competitions download -c competitive-data-science-predict-future-sales

# !kaggle competitions submit -c competitive-data-science-predict-future-sales -f sample_submission.csv -m "Message"

- **test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.**
- **The forecast should be based on combination of shop_id,item_id for the whole month of Nov 2015.**
- **Use the ID as key for the item sales (corresponding to shop_id, item_id in test.csv) and create the submission csv file.**

1. Use datatable to read the big dataset
1. Use dataprep for EDA
1. Lazypredict also for baseline
1. Optuna

In [1]:
import numpy as np
import pandas as pd
import lux

In [10]:
# !jupyter nbextension enable --py luxwidget

Enabling notebook extension luxwidget/extension...
      - Validating: ok


In [2]:
#Importing the Libraries
import datetime
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.random.seed(42)

In [3]:
# import datatable as dt

In [2]:
df = pd.read_pickle('./mypkl/merged_en.pkl')

In [3]:
type(df)

lux.core.frame.LuxDataFrame

In [ ]:
# df = df.to_pandas()
# type(df)

In [6]:
df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,2013-01-02,0,59,22154,999.0,1.0,Scene 2012 (BD),37,Movies - Blu-Ray,"Yaroslavl shopping center ""Altair"""
1,2013-04-26,3,59,944,150.0,1.0,2012 (BD),37,Movies - Blu-Ray,"Yaroslavl shopping center ""Altair"""
2,2013-06-26,5,59,944,199.5,1.0,2012 (BD),37,Movies - Blu-Ray,"Yaroslavl shopping center ""Altair"""
3,2013-07-20,6,59,944,199.5,1.0,2012 (BD),37,Movies - Blu-Ray,"Yaroslavl shopping center ""Altair"""
4,2013-09-14,8,59,944,299.0,2.0,2012 (BD),37,Movies - Blu-Ray,"Yaroslavl shopping center ""Altair"""
...,...,...,...,...,...,...,...,...,...,...
2935838,2015-10-22,33,55,13093,250.0,1.0,Map Windows paying 250 rubles [Digital Version],36,Payment card - Windows (figure),Digital storage 1C-line
2935839,2015-09-21,32,55,13091,1000.0,1.0,Map Windows Payment: 1000 rubles [Digital Vers...,36,Payment card - Windows (figure),Digital storage 1C-line
2935840,2015-09-16,32,55,13094,2500.0,1.0,Map Windows Payment: 2500 rubles [Digital Vers...,36,Payment card - Windows (figure),Digital storage 1C-line
2935841,2015-09-22,32,55,13094,2500.0,2.0,Map Windows Payment: 2500 rubles [Digital Vers...,36,Payment card - Windows (figure),Digital storage 1C-line


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935843 entries, 0 to 2935842
Data columns (total 10 columns):
 #   Column              Dtype         
---  ------              -----         
 0   date                datetime64[ns]
 1   date_block_num      Int8          
 2   shop_id             Int8          
 3   item_id             Int16         
 4   item_price          float32       
 5   item_cnt_day        float32       
 6   item_name           category      
 7   item_category_id    Int8          
 8   item_category_name  category      
 9   shop_name           category      
dtypes: Int16(1), Int8(3), category(3), datetime64[ns](1), float32(2)
memory usage: 81.9 MB


### Initial Insights
- huge df (1.6 GB) -- reduced to 73.2 MB
- 60 shops only
- skewed features, so take skewed value for all as baseline
- nothing is missing
- 

#### date and date_block_num
- High Observations on:
    - 17 Feb 2013, 16 Feb 2014, 22 Feb 2015
    - 29 Dec 2013, 28 Dec 2014
- Minorly peaks on:
    - 25 Aug 2013, 2014, 2015
- Dip in Observations on:
    - 12 May 2013, 11 May 2014, 10 May 2015

#### shop_id
- Few shops have many observations
- Few have very less

#### shop_name

#### item_id
- Sum of two gaussian distribution

#### item_name
- The largest value (Corporate package white shirt 1C Interest (34 * 42) 45 m) is over 3.33 times larger than the second largest value (Playstation Store replenishment wallet: Map payment of 1000 rubles.)
- Largest value takes only about 1% of total
- Top 10 items (by count) is about Playstation, GTA, Minecraft, Diabolo III, Call of Duty, World of War. (Games)
- Kapersky antivirus also in top 10, do people download games from unofficial sources?
- pc, ps3, ps4, xbox, jewel, essestials have many occurences (clearly gaming thing)

#### item_category_id
- about 68% of total ids's are taken by about 8-15 items
- categories 19,23,28,37-39 are medium-major in count
- categories 30,40,55 are highly frequent

#### item_category_name
- top 9 categories are about 70% of total count
- DVD, PC, Blu-ray, CD, PS4, PS3 are in the top 10 category names
- 

#### item_price
- 1 item has negative price, **drop it**
- Filled with outliers!
    - More than 99% items have price between 0 to 6100
    - 95 percentile is 2799 dollars
    
#### item_cnt_day
- 22 have negative value (0.3%) **drop it**
- 5th to 75th percentile is 1, 95th percentile is 2
    - mode value is 1 only :D
- **filter out values above 20 (as in forums)**



### Correlations
- Spearman is stronger then Pearson
- Item price is mildly negative correlated with item_id and item_category_id (may not make sense)

In [ ]:
# cleaned_df.dtypes

In [16]:
# cleaned_df = cleaned_df.to_pandas()
type(cleaned_df)

pandas.core.frame.DataFrame

In [ ]:
# ERROR WHILE SAVING PREPROCESSED DATAFRAME AS PKL
# The easiest way is to pickle it using to_pickle:
# cleaned_df.to_pickle('clean_df_new.pkl')  # where to save it, usually as a .pkl
# Then you can load it back using:
# df = pd.read_pickle(file_name)

In [17]:
cleaned_df = cleaned_df.sort_values(by = 'date')
# cleaned_df.head()

In [18]:
column_names = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price','item_cnt_day', 'item_category_id', 'item_name','item_category_name','shop_name']
cleaned_df = cleaned_df.reindex(columns=column_names)
# cleaned_df

In [19]:
my_filter = ((cleaned_df['item_price'] > 0) & (cleaned_df['item_cnt_day'] > 0) & (cleaned_df['item_cnt_day'] < 26))
cleaned_df = cleaned_df[my_filter]

- item_id (500 to 6000) are correlated with categoriy_ids 20-30
- item_id (7000 to 20,000) are correlated with categoriy_ids 38-44
- only few categories are pricey
- as time progresses, items become more pricey
    - some date block numbers have ZERO correlation at all prices. 
    - they are literally in blocks
    - lesser value of item price is correlated with all shops
- almost all categories are sold in quantity 1-2
    - only categories (71 or 72) and (28-29) are *sometimes* sold at more quantities
    - some catgories like 19, 55, 42 are strongly correlated with every date
- Feb, Aug, Dec have more sales, Nov has lesser
- day of week 4,5,6 (most probably Fri, Sat, Sun) clearly have more number of sales

In [20]:
type(cleaned_df)

pandas.core.frame.DataFrame

In [ ]:
# ERROR WHILE SAVING PREPROCESSED DATAFRAME AS PKL
# The easiest way is to pickle it using to_pickle:
# cleaned_df.to_pickle('clean_df_today')  # where to save it, usually as a .pkl
# Then you can load it back using:
# df = pd.read_pickle(file_name)

In [ ]:
# cleaned_df.to_pickle('clean_df_today')  # where to save it, usually as a .pkl
# cleaned_df.to_csv('clean_data.csv')

In [21]:
df = cleaned_df.copy()
del cleaned_df

In [ ]:
# df = pd.read_pickle('clean_df_today')

In [ ]:
df.info()

### feature engineering on datetime features

In [22]:
from fast_ml.feature_engineering import FeatureEngineering_DateTime

In [23]:
dt_fe = FeatureEngineering_DateTime()
dt_fe.fit(df, datetime_variables=['date'], prefix= 'date_')
df = dt_fe.transform(df)
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,item_name,item_category_name,shop_name,...,date_is_quarter_end,date_is_quarter_start,date_is_year_end,date_is_year_start,date_time,date_hour,date_minute,date_second,date_is_weekend,date_day_part
148963,2013-01-01,0,27,8665,299.0,1,40,БАРБОСКИНЫ ВЫПУСК 5: УЧЕНИК ЧАРОДЕЯ,Кино - DVD,"Москва ТЦ ""МЕГА Белая Дача II""",...,False,True,False,True,00:00:00,0,0,0,0,None
177545,2013-01-01,0,27,10324,1199.0,1,38,ГАРРИ ПОТТЕР И ДАРЫ СМЕРТИ. ЧАСТЬ 1 (3D BD),Кино - Blu-Ray 3D,"Москва ТЦ ""МЕГА Белая Дача II""",...,False,True,False,True,00:00:00,0,0,0,0,None
177549,2013-01-01,0,27,11222,1699.0,1,38,ДРУГОЙ МИР : ПРОБУЖДЕНИЕ (3D BD),Кино - Blu-Ray 3D,"Москва ТЦ ""МЕГА Белая Дача II""",...,False,True,False,True,00:00:00,0,0,0,0,None
177579,2013-01-01,0,27,11076,749.5,1,38,ДИКИЙ ОКЕАН (3D BD+BD),Кино - Blu-Ray 3D,"Москва ТЦ ""МЕГА Белая Дача II""",...,False,True,False,True,00:00:00,0,0,0,0,None
923815,2013-01-01,0,54,15115,170.0,2,67,Мини-головоломка 11***/ Mini Wire 11*** Арт. 4...,Подарки - Развитие,"Химки ТЦ ""Мега""",...,False,True,False,True,00:00:00,0,0,0,0,None


In [ ]:
df.columns

In [24]:
df[['date_year', 'date_quarter', 'date_month', 'date_day',
       'date_day_of_week', 'date_day_of_year', 'date_weekofyear',
       'date_is_month_end', 'date_is_month_start', 'date_is_quarter_end',
       'date_is_quarter_start', 'date_is_year_end', 'date_is_year_start',
       'date_time', 'date_hour', 'date_minute', 'date_second',
       'date_is_weekend', 'date_day_part']].head()

,date_year,date_quarter,date_month,date_day,date_day_of_week,date_day_of_year,date_weekofyear,date_is_month_end,date_is_month_start,date_is_quarter_end,date_is_quarter_start,date_is_year_end,date_is_year_start,date_time,date_hour,date_minute,date_second,date_is_weekend,date_day_part
148963,2013,1,1,1,1,1,1,False,True,False,True,False,True,00:00:00,0,0,0,0,None
177545,2013,1,1,1,1,1,1,False,True,False,True,False,True,00:00:00,0,0,0,0,None
177549,2013,1,1,1,1,1,1,False,True,False,True,False,True,00:00:00,0,0,0,0,None
177579,2013,1,1,1,1,1,1,False,True,False,True,False,True,00:00:00,0,0,0,0,None
923815,2013,1,1,1,1,1,1,False,True,False,True,False,True,00:00:00,0,0,0,0,None


In [ ]:
# _, cleaned_df_new = clean_df(df, remove_duplicate_entries = True)
# ERROR for boolean values

In [ ]:
# df.to_pickle('clean_df_today')

# df = pd.read_pickle('clean_df_today')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2926186 entries, 148963 to 1630139
Data columns (total 29 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   date                   datetime64[ns]
 1   date_block_num         Int8          
 2   shop_id                Int8          
 3   item_id                Int16         
 4   item_price             float32       
 5   item_cnt_day           Int16         
 6   item_category_id       Int8          
 7   item_name              string        
 8   item_category_name     string        
 9   shop_name              string        
 10  date_year              int64         
 11  date_quarter           int64         
 12  date_month             int64         
 13  date_day               int64         
 14  date_day_of_week       int64         
 15  date_day_of_year       int64         
 16  date_weekofyear        int64         
 17  date_is_month_end      bool          
 18  date_is_month_sta

feature ideas:
- date since last sale of shop
- date since last sale of item
- date since last sale of customer
- days till holiday
- **is_holiday??**
- other examples of difference between dates

In [26]:
# convert boolean to integers
bool_cols = ['date_is_month_end', 'date_is_month_start', 'date_is_quarter_end', 
             'date_is_quarter_start','date_is_year_end', 'date_is_year_start']
df[bool_cols] = df[bool_cols]*1
df[bool_cols].head(2)

,date_is_month_end,date_is_month_start,date_is_quarter_end,date_is_quarter_start,date_is_year_end,date_is_year_start
148963,0,1,0,1,0,1
177545,0,1,0,1,0,1


In [27]:
nunique = df.nunique()
cols_to_drop = nunique[nunique == 1].index
cols_to_drop

Index(['date_time', 'date_hour', 'date_minute', 'date_second'], dtype='object')

In [28]:
df.drop(cols_to_drop, axis=1, inplace = True)
# df.columns

In [ ]:
# inferred_dtypes, cleaned_df = clean_df(df, remove_duplicate_entries = True, downcast_memory = True)
# still NA values error

In [32]:
import gc

In [33]:
def downcast_df_int_columns(df):
    list_of_columns = list(df.select_dtypes(include=["int32", "int64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        
        for col in list_of_columns:
            df[col] = pd.to_numeric(df[col], downcast="integer")

#     else:
#         print("no columns to downcast")
    gc.collect()
    print("done")
    

In [34]:
def downcast_df_float_columns(df):
    list_of_columns = list(df.select_dtypes(include=["float64"]).columns)
        
    if len(list_of_columns)>=1:
        max_string_length = max([len(col) for col in list_of_columns]) # finds max string length for better status printing
        
        for col in list_of_columns:
            df[col] = pd.to_numeric(df[col], downcast="float")
#     else:
#         print("no columns to downcast")
    gc.collect()
    print("done")
    

In [35]:
downcast_df_int_columns(df)
downcast_df_float_columns(df)

done
done


In [ ]:
df.info()

In [39]:
# check if date is holiday or in holiday week (let me make it as +- 3 days)
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# dr = df['date']
start_observations = (df['date']-pd.Timedelta(days = 30)).min()
end_observations = (df['date']+ pd.Timedelta(days = 90)).max()
# important because test set is one month (30 days) after training thing
# if simply max of training date taken, then 'until_holiday' for last date will have NaN

In [40]:
cal = calendar()
holidays = cal.holidays(start = start_observations, end = end_observations)
holidays

DatetimeIndex(['2012-12-25', '2013-01-01', '2013-01-21', '2013-02-18',
               '2013-05-27', '2013-07-04', '2013-09-02', '2013-10-14',
               '2013-11-11', '2013-11-28', '2013-12-25', '2014-01-01',
               '2014-01-20', '2014-02-17', '2014-05-26', '2014-07-04',
               '2014-09-01', '2014-10-13', '2014-11-11', '2014-11-27',
               '2014-12-25', '2015-01-01', '2015-01-19', '2015-02-16',
               '2015-05-25', '2015-07-03', '2015-09-07', '2015-10-12',
               '2015-11-11', '2015-11-26', '2015-12-25', '2016-01-01',
               '2016-01-18'],
              dtype='datetime64[ns]', freq=None)

In [41]:
df['is_Holiday'] = df['date'].isin(holidays)
df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,item_name,item_category_name,shop_name,...,date_weekofyear,date_is_month_end,date_is_month_start,date_is_quarter_end,date_is_quarter_start,date_is_year_end,date_is_year_start,date_is_weekend,date_day_part,is_Holiday
148963,2013-01-01,0,27,8665,299.0,1,40,БАРБОСКИНЫ ВЫПУСК 5: УЧЕНИК ЧАРОДЕЯ,Кино - DVD,"Москва ТЦ ""МЕГА Белая Дача II""",...,1,0,1,0,1,0,1,0,None,True
177545,2013-01-01,0,27,10324,1199.0,1,38,ГАРРИ ПОТТЕР И ДАРЫ СМЕРТИ. ЧАСТЬ 1 (3D BD),Кино - Blu-Ray 3D,"Москва ТЦ ""МЕГА Белая Дача II""",...,1,0,1,0,1,0,1,0,None,True


In [42]:
# DateOffsets (Day, Hour, Minute, Second, Milli, Micro, Nano) can also be used in construction.

# pd.Timedelta(pd.offsets.Day(1))
# Out[15]: Timedelta('0 days 00:00:02')
after_list = []
for day in [-3,-2,-1,1,2,3]:
    near_holiday_days = [i + pd.Timedelta(days = day) for i in holidays]
    after_list.append(near_holiday_days) 

In [43]:
after_list = np.array(after_list).ravel()
after_list[:5]

array([Timestamp('2012-12-22 00:00:00'), Timestamp('2012-12-29 00:00:00'),
       Timestamp('2013-01-18 00:00:00'), Timestamp('2013-02-15 00:00:00'),
       Timestamp('2013-05-24 00:00:00')], dtype=object)

In [44]:
df['is_near_Holiday'] = df['date'].isin(after_list)
df.head(2)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,item_name,item_category_name,shop_name,...,date_is_month_end,date_is_month_start,date_is_quarter_end,date_is_quarter_start,date_is_year_end,date_is_year_start,date_is_weekend,date_day_part,is_Holiday,is_near_Holiday
148963,2013-01-01,0,27,8665,299.0,1,40,БАРБОСКИНЫ ВЫПУСК 5: УЧЕНИК ЧАРОДЕЯ,Кино - DVD,"Москва ТЦ ""МЕГА Белая Дача II""",...,0,1,0,1,0,1,0,None,True,False
177545,2013-01-01,0,27,10324,1199.0,1,38,ГАРРИ ПОТТЕР И ДАРЫ СМЕРТИ. ЧАСТЬ 1 (3D BD),Кино - Blu-Ray 3D,"Москва ТЦ ""МЕГА Белая Дача II""",...,0,1,0,1,0,1,0,None,True,False


In [45]:
df['is_near_Holiday'].value_counts()

False    2385757
True      540429
Name: is_near_Holiday, dtype: int64

In [46]:
df.to_pickle('clean_df_today')

In [47]:
df = pd.read_pickle('clean_df_today')

In [48]:
holiday_df = pd.DataFrame({'date1':holidays})
holiday_df.head()

,date1
0,2012-12-25
1,2013-01-01
2,2013-01-21
3,2013-02-18
4,2013-05-27


In [49]:
df = pd.merge_asof(df, holiday_df, left_on='date', right_on='date1', direction='forward')
# df

In [50]:
df = pd.merge_asof(df, holiday_df, left_on='date', right_on='date1')
# df

In [51]:
df['until_holiday'] = df.pop('date1_x').sub(df['date']).dt.days
df['since_holiday'] = df['date'].sub(df.pop('date1_y')).dt.days

In [52]:
df[['until_holiday', 'since_holiday']]

,until_holiday,since_holiday
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
2926181,11,19
2926182,11,19
2926183,11,19
2926184,11,19


In [53]:
(df['until_holiday']*(-1) == df['since_holiday']).value_counts()
# sab until aur since negative pairs hai

False    2863852
True       62334
dtype: int64

In [ ]:
# trial example
# dff = pd.DataFrame()
# dff["date"] = pd.date_range("2015", periods=5)

# holidayss = ["30.12.2014", "02.01.2015", "05.01.2015", "10.01.2015"]
# holidayss = pd.to_datetime(holidayss, dayfirst=True)

# df1 = pd.DataFrame({'date1':holidayss})

# dff = pd.merge_asof(dff, df1, left_on='date', right_on='date1', direction='forward')
# dff = pd.merge_asof(dff, df1, left_on='date', right_on='date1')

# dff['until'] = dff.pop('date1_x').sub(dff['date']).dt.days
# dff['since'] = dff['date'].sub(dff.pop('date1_y')).dt.days
# dff

# date	until	since
# 0	2015-01-01	1	2
# 1	2015-01-02	0	0
# 2	2015-01-03	2	1
# 3	2015-01-04	1	2
# 4	2015-01-05	0	0

In [ ]:
# df.info()

In [54]:
df['date_day_part'].value_counts() # remove this since all are None values
df.drop('date_day_part', axis=1, inplace = True)
# df.columns

In [55]:
df[['is_Holiday', 'is_near_Holiday']] = df[['is_Holiday', 'is_near_Holiday']]*1

In [56]:
downcast_df_int_columns(df)
downcast_df_float_columns(df)

done
done


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2926186 entries, 0 to 2926185
Data columns (total 28 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   date                   datetime64[ns]
 1   date_block_num         Int8          
 2   shop_id                Int8          
 3   item_id                Int16         
 4   item_price             float32       
 5   item_cnt_day           Int16         
 6   item_category_id       Int8          
 7   item_name              string        
 8   item_category_name     string        
 9   shop_name              string        
 10  date_year              int16         
 11  date_quarter           int8          
 12  date_month             int8          
 13  date_day               int8          
 14  date_day_of_week       int8          
 15  date_day_of_year       int16         
 16  date_weekofyear        int8          
 17  date_is_month_end      int8          
 18  date_is_month_start   

In [58]:
df.to_pickle('clean_df_today')

In [59]:
gc.collect()

0

In [60]:
df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,item_name,item_category_name,shop_name,...,date_is_month_start,date_is_quarter_end,date_is_quarter_start,date_is_year_end,date_is_year_start,date_is_weekend,is_Holiday,is_near_Holiday,until_holiday,since_holiday
0,2013-01-01,0,27,8665,299.0,1,40,БАРБОСКИНЫ ВЫПУСК 5: УЧЕНИК ЧАРОДЕЯ,Кино - DVD,"Москва ТЦ ""МЕГА Белая Дача II""",...,1,0,1,0,1,0,1,0,0,0
1,2013-01-01,0,27,10324,1199.0,1,38,ГАРРИ ПОТТЕР И ДАРЫ СМЕРТИ. ЧАСТЬ 1 (3D BD),Кино - Blu-Ray 3D,"Москва ТЦ ""МЕГА Белая Дача II""",...,1,0,1,0,1,0,1,0,0,0
2,2013-01-01,0,27,11222,1699.0,1,38,ДРУГОЙ МИР : ПРОБУЖДЕНИЕ (3D BD),Кино - Blu-Ray 3D,"Москва ТЦ ""МЕГА Белая Дача II""",...,1,0,1,0,1,0,1,0,0,0
3,2013-01-01,0,27,11076,749.5,1,38,ДИКИЙ ОКЕАН (3D BD+BD),Кино - Blu-Ray 3D,"Москва ТЦ ""МЕГА Белая Дача II""",...,1,0,1,0,1,0,1,0,0,0
4,2013-01-01,0,54,15115,170.0,2,67,Мини-головоломка 11***/ Mini Wire 11*** Арт. 4...,Подарки - Развитие,"Химки ТЦ ""Мега""",...,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2926181,2015-10-31,33,47,4367,1399.0,1,19,"LEGO Star Wars III: the Clone Wars [PS3, русск...",Игры - PS3,"Сургут ТРЦ ""Сити Молл""",...,0,0,0,0,0,1,0,0,11,19
2926182,2015-10-31,33,56,15040,599.0,1,28,"Метро 2033. Возвращение [PC, русская версия]",Игры PC - Дополнительные издания,"Чехов ТРЦ ""Карнавал""",...,0,0,0,0,0,1,0,0,11,19
2926183,2015-10-31,33,57,3327,149.0,1,21,"FIFA 13 [PSP, русская версия]",Игры - PSP,"Якутск Орджоникидзе, 56",...,0,0,0,0,0,1,0,0,11,19
2926184,2015-10-31,33,16,3243,599.0,3,30,"Evolve [PC, русская версия]",Игры PC - Стандартные издания,"Коломна ТЦ ""Рио""",...,0,0,0,0,0,1,0,0,11,19
